In [1]:
import pandas as pd
import sklearn.cluster as cluster
import httrees as ht
import pickle

12/07/2021 11:17:04 AM adding document #0 to Dictionary(0 unique tokens: [])
12/07/2021 11:17:04 AM built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
12/07/2021 11:17:04 AM Dictionary lifecycle event {'msg': "built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)", 'datetime': '2021-12-07T11:17:04.854564', 'gensim': '4.1.2', 'python': '3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}



`httrees` is a Python module for hierarchical topic modeling, providing several text vectorizers to represent your text data, and a hierarchical clustering algorithm meant to operate over said text representations that utilizes multiple flat clusterers. In that sense the model it implements it is actually a general hierarchical clustering algorithm, though the surrounding utilities are tailored towards text data.

This notebook serves as a minimal demonstration of the capabilities of the `httrees` module, and how to use it to perform hierarchical topic modeling on a collection of text documents. We will also discuss implementation details at a high level. The source is available for further perusal on [GitHub](https://github.com/bllguo/CourseProject).

We will use a publically available dataset of Amazon product reviews from [Kaggle](https://www.kaggle.com/kashnitsky/flat-hierarchical-tf-idf-logreg-baseline/data) that contains review text, up to three levels of categorization, and other features that are not relevant for our purposes.

In [2]:
df = pd.read_csv('train_40k.csv')
df = df[['Text', 'Cat1', 'Cat2', 'Cat3']]
df.head()

,Text,Cat1,Cat2,Cat3
0,The description and photo on this product need...,grocery gourmet food,meat poultry,jerky
1,This was a great book!!!! It is well thought t...,toys games,games,unknown
2,"I am a first year teacher, teaching 5th grade....",toys games,games,unknown
3,I got the book at my bookfair at school lookin...,toys games,games,unknown
4,Hi! I'm Martine Redman and I created this puzz...,toys games,puzzles,jigsaw puzzles


# Text representations: `httrees.vectorizer`

Several vectorizers are available in `httrees.vectorizer`:
- `CountVectorizer`: Simple bag-of-words vectorizer.
- `TfidfVectorizer`: Term frequency-inverse document frequency vectorizer.
- `EmbeddingVectorizer`: Vectorizer that uses a pretrained set of word embeddings to represent text. Documents are represented as the average of the embeddings of their words. Unknown words are either skipped or represented with an `<UNK>` embedding.
- `KVVectorizer`: Similar to `EmbeddingVectorizer`, the `KVVectorizer` assumes embeddings are provided as `gensim.models.KeyedVectors` instead of a generic key-value store, which allows it to handle unknown words through `KeyedVectors` methods. For instance, for `FastText` embeddings, unknown words will be vectorized using learned character embeddings.

They follow the sklearn API. For example, `TfidfVectorizer` can be used like so:

In [3]:
vectorizer = ht.vectorizer.TfidfVectorizer(unk=True, threshold=15)    # words that appear <= 15 times are mapped to UNK
vectorizer.fit(df['Text'])

In [4]:
vectorizer.predict(df['Text'])

array([[0.9214465 , 0.03934427, 0.21747493, ..., 0.        , 0.        ,
        0.        ],
       [1.44499565, 0.04113264, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.66926114, 0.00952545, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.58949521, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.93894232, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

The vocabulary is available in the `vocabulary` attribute of the vectorizer.

In [5]:
for word, i in vectorizer.vocabulary.items():
    print(word, i)
    if i > 4:
        break

UNK 0
The 1
description 2
and 3
photo 4
on 5


Similarly, the term frequencies are available in the `term_freqs` attribute.

In [6]:
i = 0
for word, cnt in vectorizer.term_freqs.items():
    print(word, cnt)
    i += 1
    if i > 4:
        break

The 16182
description 268
and 94682
photo 89
on 23067


Unlike the count-based vectorizers, `EmbeddingVectorizer` and `KVVectorizer`'s `fit` methods don't do anything. Instead, pretrained word vectors are to be passed in on initialization. `EmbeddingVectorizer` takes a `dict` or any other generic key-value store, and `KVVectorizer` takes a `gensim.models.KeyedVectors` object.

The below code downloads a set of pretrained word embeddings from `gensim`:

In [7]:
import gensim.downloader as api
fname = 'word2vec-google-news-300'
model = api.load(fname)
embeddings = {k: model.get_vector(k) for k in model.index_to_key}

12/07/2021 11:17:12 AM loading projection weights from C:\Users\bllgu/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
12/07/2021 11:17:51 AM KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from C:\\Users\\bllgu/gensim-data\\word2vec-google-news-300\\word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2021-12-07T11:17:51.417799', 'gensim': '4.1.2', 'python': '3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'load_word2vec_format'}


In [8]:
type(model)

gensim.models.keyedvectors.KeyedVectors

Downloaded `gensim` models can be passed directly to a `KVVectorizer`, but we will proceed with an `EmbeddingVectorizer` here.

We can also just load from local disk:

In [9]:
with open('glove_wiki_300.pkl', 'rb') as f:
    embeddings = pickle.loads(f.read())

In [10]:
e_vectorizer = ht.vectorizer.EmbeddingVectorizer(embeddings)
X = e_vectorizer.predict(df['Text'])
X[0]

array([-1.33335047e-01,  1.78493317e-01, -1.01416264e-01, -2.18229094e-01,
       -1.48588412e-01,  1.13462319e-01, -9.58337719e-02,  7.88709108e-02,
        8.27962819e-02, -1.59075406e+00,  1.04738616e-02, -5.22186992e-02,
       -5.72863122e-02,  1.00738781e-01,  5.79180455e-02,  6.72442959e-02,
       -1.33164410e-01,  4.28178940e-05, -5.71114581e-02, -2.29855500e-01,
       -1.50674365e-01,  2.44362183e-01,  7.13207257e-02,  2.52037861e-01,
       -2.06734137e-01, -2.66635476e-02,  2.24481912e-03, -4.50084562e-02,
       -8.28522278e-02,  6.41427917e-02,  1.46519584e-02,  2.06074228e-01,
       -2.16876966e-01,  1.75746591e-01, -6.97974121e-01,  1.47815000e-01,
        9.90912728e-02, -1.65078191e-02, -7.06567724e-02, -1.03543996e-02,
       -6.87062176e-02, -5.48620958e-02, -7.18452754e-02,  7.64035252e-02,
        8.73894112e-02,  4.18272725e-02,  4.66988709e-02,  7.64650906e-02,
       -1.70375043e-01,  4.85615024e-02,  3.88974835e-02, -1.07020046e-01,
       -5.66541356e-02,  

We can also fine-tune our embeddings on our data, adding new words to the vocabulary, and updating the embeddings of existing words. This process is detailed in [a separate notebook here](https://github.com/bllguo/CourseProject/blob/main/example_finetune.ipynb).

# Running the model: `httrees.models`

Now that we have transformed our data with a vectorizer, we can use the `httrees.models` module to run a hierarchical clustering algorithm. 

The `TopicTreeModel` builds a tree representation of the topic hierarchy. 

First, we describe the tree itself. In this tree, every node is a topic/cluster. A node can have any number of children, each of which represent a subtopic underneath it. A `Node` has the following attributes:
- depth: The depth of the node in the tree. This is important for the algorithm as the depth determines which clustering model to use.
- mask: Every node stores a boolean mask that indicates which documents in the training data are assigned to it. A child node's mask will be a subset of the parent's mask.
- center: Once the model has been fit, the center of the node represents the cluster center, typically meaning an average of the assigned documents.
- children: A list of child nodes.

The `TopicTreeModel` builds this tree according to a given configuration:

In [13]:
config = [(cluster.MiniBatchKMeans, {'validate': {'n_clusters': {'values': [8, 10]}}}),
          (cluster.MeanShift, {}),
          (cluster.AgglomerativeClustering, 
           {'n_clusters': 3,
            'affinity': 'euclidean', 
            'linkage': 'ward'})
          ]

There are 3 elements in this example config, meaning that the hierarchy will be 3 levels deep. If only 2 elements are in the list, the hierarchy will be 2 levels deep, and so on. Each element contains a clustering model, like `cluster.MiniBatchKMeans`, and a dictionary containing the parameters for that clustering model. See [sklearn documentation](https://scikit-learn.org/stable/modules/clustering.html#) for some other compatible models. `httrees` was tested on kmeans, affinity propagation, mean shift, and agglomerative clustering.

The `validate` parameter, seen above as an input for `MiniBatchKMeans`, is not a native sklearn param. It is a dict of model params that the `TopicTreeModel` will evaluate to find the best choice. This allows for usage of clustering models where `n_clusters` must be specified, for example. In the above config, 8 and 10 clusters will be tested for the highest level category. Details on how this validation is performed is located [in the source](https://github.com/bllguo/CourseProject/blob/main/httrees/models.py#L82-L109). BE VERY CAREFUL when using this parameter, as the validation computation is very costly (it computes pairwise cosine similarities between all documents in a cluster).

For every node at a given depth in the tree, one instance of the model corresponding to that depth will be fit. For instance, in this case, `MiniBatchKMeans` will be fit at the first level to generate the highest-level topics. This will be saved in the root node. In this example, we might generate very broad topics/clusters corresponding to Amazon categories like 'Electronics', 'Home & Garden', and 'Toys & Games'.

These clusters will be child nodes of the root node. For every cluster, a `MeanShift` model will be fit to generate 3rd level subtopics. For instance, the `MeanShift` model for the 'Electronics' cluster could generate 3 subtopics corresponding to 'Cell Phones & Accessories', 'Computers', and 'Tablets'. These will be child nodes of the 'Electronics' node. A separate `MeanShift` model will be fit for `Home & Garden`, etc.

This continues until the max depth specified by the config - in this case, 3.

In [14]:
tree = ht.models.TopicTreeModel(config)
tree.fit(X)

The `TopicTreeModel` provides a `decode` method that returns the cluster assignments as an array of shape (`X.shape[0]`, `len(config)`) - a row for every document - as well as a summary of each cluster.

In [15]:
assignments, clusters = tree.decode()
assignments

array([[   3,  733, 3404],
       [   1,    9, 2680],
       [   6, 1537, 4208],
       ...,
       [   2,  693, 3364],
       [   4, 1208, 3879],
       [   7, 2023, 4694]])

In [16]:
clusters[8]

{'topics': [],
 'center': array([-1.05563528e-01,  9.60578198e-02, -6.02419291e-02, -1.47723918e-01,
        -4.84223794e-02, -1.32323935e-03, -4.77894796e-02,  3.68519057e-02,
         1.08842833e-01, -1.56762253e+00,  1.05297648e-01, -2.96364271e-02,
        -6.62311710e-02,  1.14088915e-01,  1.26955661e-02,  8.62555348e-02,
        -1.54350493e-01,  4.17050668e-03, -6.45460731e-03, -1.99513326e-02,
         2.74512330e-02,  2.46578148e-01,  1.05359679e-01,  1.08188672e-01,
        -2.49874049e-01, -6.18180829e-02,  3.47082471e-02, -1.34118283e-01,
        -5.09657673e-02,  5.69302300e-02, -1.61627532e-02,  2.53710563e-01,
        -2.28054691e-01, -1.61259225e-02, -8.50624737e-01,  1.69397810e-01,
        -8.06885621e-02,  2.08911511e-02, -6.97873893e-02,  4.03181623e-02,
         2.03055585e-02, -1.42359836e-01, -4.31400195e-02, -5.23352777e-02,
         1.28550935e-01,  9.26878994e-02,  2.07890434e-01,  1.30938008e-01,
        -6.26850417e-02,  5.77091664e-02,  7.57689740e-02, -1.3

If provided the original texts and vectorizer, `decode` will also attempt to do some summarization and associate each `cluster` with a topic string. This is very experimental and doesn't handle noise words well - it is based on processing the most common bigrams in each cluster - so it will perform poorly on longer documents.

This can be turned into a dataframe through the `flatten` method.

In [18]:
assignments, clusters = tree.decode(df['Text'], vectorizer=e_vectorizer)

In [19]:
out = tree.flatten(df['Text'], assignments, clusters)
out.head(10)

,Text,level1,level2,level3
0,The description and photo on this product need...,"[to be, to the, and the]","[to be, to the, and the]","[to be, to the, and the]"
1,This was a great book!!!! It is well thought t...,"[This is, it is, It is]","[It is, it is, and it]","[It is, it is, and it]"
2,"I am a first year teacher, teaching 5th grade....","[to the, and the, and I]","[to the, and the, and I]","[to the, and the, and I]"
3,I got the book at my bookfair at school lookin...,"[and I, and it, I would]","[and I, and it, I would]","[and I, and it, I would]"
4,Hi! I'm Martine Redman and I created this puzz...,"[I have, is the, and the]","[I have, is the, and the]","[I have, is the, and the]"
5,"My eight year old loves this game, whenever he...","[this for, for my, and I]","[this for, for my, and I]","[this for, for my, and I]"
6,The real joy of this movie doesn't lie in its ...,"[to be, to the, and the]","[to be, to the, and the]","[to be, to the, and the]"
7,"Okay, Tim Burton is genuine. He haunts you wit...","[This is, it is, It is]","[It is, it is, and it]","[It is, it is, and it]"
8,"Boundaries, along with counseling, has given m...","[This is, it is, It is]","[It is, it is, and it]","[It is, it is, and it]"
9,120 colors? I say 120 sticks of fun! And a fre...,"[This is, it is, It is]","[It is, it is, and it]","[It is, it is, and it]"
